<a href="https://colab.research.google.com/github/ericzhang789/mortgage-tools/blob/bokeh/mortgage_payment_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import Dependencies
%%capture
!pip install pandas numpy numpy_financial matplotlib bokeh
import math
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show 

from bokeh.models import CustomJS, ColumnDataSource, HoverTool, Div, Select, TextInput, CheckboxGroup, Spinner
from bokeh.plotting import figure
output_notebook()


In [2]:
#@title Define Functions
cmhc_rates = {'5':0.04, '10':0.031, '15':0.028, '20':0.00}
PST = 0.08

def get_mortgage_amt(purchase_price, percent_down):
  purchase_price = 576000
  percent_down = '10'

  #apply downpayment
  mortgage_amount = purchase_price*(1-float(percent_down)/100)

  #add mortgage insurance if applicable
  mortgage_amount += mortgage_amount*cmhc_rates[percent_down]
  return mortgage_amount


def payment_schedule(interest_rate, payment_frequency, amortization_years, mortgage_amount):
  pmt = npf.pmt(interest_rate/payment_frequency, payment_frequency*amortization_years, mortgage_amount)
  period = np.arange(payment_frequency*amortization_years)+1
  ipmt = npf.ipmt(interest_rate/payment_frequency, period, amortization_years*payment_frequency, mortgage_amount)
  ppmt = npf.ppmt(interest_rate/payment_frequency, period, amortization_years*payment_frequency, mortgage_amount)
  #np.allclose(ipmt+ppmt, pmt)

  principal = mortgage_amount
  pcpl = []
  year = []
  fmt = '{0:2d} {1:8.2f} {2:8.2f} {3:8.2f}'

  for payment in period:
      index = payment - 1
      principal = principal + ppmt[index]
      pcpl.append(principal)
      year.append(math.ceil(period[index]/26))
      #print(fmt.format(payment, ppmt[index], ipmt[index], principal))

  #format nicely
  pcpl = np.around(pcpl, 2)
  pmt = np.around(pmt, 2)
  ipmt = np.around(ipmt, 2)
  ppmt = np.around(ppmt, 2)

  #arrange in dict then return df
  pmt_dict = {'pmt_num': abs(period), 'year': year, 'pmt': abs(pmt), 'ppmt': abs(ppmt), 'ipmt': abs(ipmt), 'pcpl_rem': abs(pcpl)}
  return pd.DataFrame(data = pmt_dict)

In [3]:
#@markdown Please enter mortgage terms:
purchase_price = 576000 #@param {type:"number"}
percent_down = '10' #@param ["10", "15", "20"]
interest_rate = 1.68 #@param {type:"number"}
amortization_years = 25 #@param {type:"slider", min:0, max:30, step:1}
payment_frequency = 'bi-weekly' #@param ["bi-weekly", "monthly"]
export_csv = False #@param {type:"boolean"}

#process mortgage amount
mortgage_amount = get_mortgage_amt(purchase_price, percent_down)

#cleanse user input
if payment_frequency == 'bi-weekly':
  payment_frequency_num = 26
elif payment_frequency == 'monthly':
  payment_frequency_num = 12

interest_rate = interest_rate/100
payment_schedule_df = payment_schedule(interest_rate, payment_frequency_num, amortization_years, mortgage_amount)

if export_csv:
  payment_schedule_df.to_csv('mortgage_schedule.csv', index=False)
else:
  print(payment_schedule_df.head())

   pmt_num  year      pmt    ppmt    ipmt   pcpl_rem
0        1     1  1007.25  661.90  345.35  533808.50
1        2     1  1007.25  662.33  344.92  533146.17
2        3     1  1007.25  662.76  344.49  532483.41
3        4     1  1007.25  663.18  344.07  531820.23
4        5     1  1007.25  663.61  343.64  531156.62


In [4]:
#prototyping with bokeh

#last payment of year
#group = payment_schedule_df.groupby('year').min('pcpl_rem')

#mean payment per year
group = payment_schedule_df.groupby('year').mean().round(2)

pmt_str = str(group.pmt.max()) + ' ' + payment_frequency

TOOLTIPS = [('Principal', '@ppmt'), ('Interest', '@ipmt')]

#TODO: payment / frequency in title
p = figure(plot_width = 900, x_axis_label='Years', y_axis_label='Payment Amount per Period ($)', 
           title = 'Average Mortgage Payment Breakdown per Year ($' + pmt_str+')',
           tooltips=TOOLTIPS
           )
p.vbar_stack(['ppmt', 'ipmt'], x='year', source = group, width=0.9, 
             color=['blue', 'orange'], legend_label=['Principal', 'Interest'])
p.y_range.start = 0
p.y_range.range_padding = 0.25
p.x_range.start = 0
p

p.legend.location = 'top_left'
p.legend.orientation = 'horizontal'
show(p)

In [24]:
def inputs_callback(attr, old, new):
  input_list = ['purchase_price', 'percent_down', 'interest_rate', 'amortization_years', 'payment_frequency']
  if int(num_scenarios.value) == 2: 

In [25]:
#I think these controls could be wrapped in a div with SCENARIO X as titles

#prototype input controls
num_scenarios = Select(title='Select the number of scenarios to compare', value = '2', options = ['1', '2', '3', '4'])

#below may need to be stored as a dict
purchase_prices = TextInput(title='Home Purchase Price', value ='576000')
percent_down = Select(title='Percent Down %', value = '10', options=['10', '15', '20'])
interest_rate = Spinner(title='Interest Rate %', low = 0.00, high = 10.00, step = 0.01, value = 1.68)
amortization_years = Spinner(title='Amortization (Years)', low = 5, high = 30, step = 5,value = 25)
payment_frequency = Select(title='Payment Frequency', value = 'monthly', options=['bi-weekly', 'monthly'])
#something here for downloading reports later

show(num_scenarios)
num_scenarios.on_change('value', inputs_callback)



#controls = [purchase_prices, percent_down, interest_rate, amortization_years, payment_frequency]
#for control in controls:
#  show(control)

In [28]:
num_scenarios.value

'2'